In [16]:
# dependencies

import os
import pandas as pd
import numpy as np
import json
import pygeoj
import requests

# load geojson
filepath = os.path.join("Data/production_export.geojson")
with open(filepath) as jsonfile:
    congress_gjs = json.load(jsonfile)


In [17]:
# Public Open Data
data = pygeoj.load("Data/production_export.geojson")

type_raw = []
coordinates = []
state_label = []
district = []
rep_sen = []
party = []
name = []
url = []
state_code = []
geoid = []

for feature in data:
    try:
        type_raw.append(feature.geometry.type)
        coordinates.append(feature.geometry.coordinates)
        state_label.append(feature.properties['state_label'])
        district.append(feature.properties['district'])
        rep_sen.append(feature.properties['rep_sen'])
        party.append(feature.properties['party'])
        name.append(feature.properties['name'])
        url.append(feature.properties['url'])
        state_code.append(feature.properties['state_code'])
        geoid.append(feature.properties['geoid'])
    except (TypeError, IndexError):
        print("Type Error, Traceback (most recent call last) ... skipping")
    except (KeyError, IndexError):
        print("Key Error, Traceback (most recent call last) ... skipping")


Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping
Key Error, Traceback (most recent call last) ... skipping


In [18]:
type_final = []
for val in type_raw:
    if val != 'Null':
        type_final.append(val)


In [19]:
data_dict = {'type' : type_final, 'coordinates': coordinates, 'state_label': state_label, 'district': district, \
             'rep_sen': rep_sen, 'party': party, 'name': name, 'url': url, 'state_code': state_code, 'geoid': geoid}


In [20]:
data_df = pd.DataFrame(data_dict)
data_df.head()


,type,coordinates,state_label,district,rep_sen,party,name,url,state_code,geoid
0,Polygon,"[[[-88.202959, 35.008028], [-88.000032, 35.005...",Alabama,05,Representative,Republican,"Brooks, Mo",https://www.congress.gov/member/mo-brooks/B001274,01,0105
1,Polygon,"[[[-112.147914, 33.304647], [-112.100437, 33.3...",Arizona,09,Representative,Democratic,"Sinema, Kyrsten",https://www.congress.gov/member/kyrsten-sinema...,04,0409
2,MultiPolygon,"[[[[-122.520852, 37.594178], [-122.515326, 37....",California,14,Representative,Democratic,"Speier, Jackie",https://www.congress.gov/member/jackie-speier/...,06,0614
3,Polygon,"[[[-123.056115, 39.048815], [-123.050849, 39.0...",California,05,Representative,Democratic,"Thompson, Mike",https://www.congress.gov/member/mike-thompson/...,06,0605
4,MultiPolygon,"[[[[-122.446316, 37.861046], [-122.441271, 37....",California,02,Representative,Democratic,"Huffman, Jared",https://www.congress.gov/member/jared-huffman/...,06,0602


In [260]:
data_df[data_df["geoid"] == "2801"]

,type,coordinates,state_label,district,rep_sen,party,name,url,state_code,geoid
26,Polygon,"[[[-90.310005, 34.875097], [-90.301957, 34.880...",Mississippi,01,Representative,Republican,"Kelly, Trent",https://www.congress.gov/member/trent-kelly/K0...,28,2801
224,Polygon,"[[[-90.310005, 34.875097], [-90.301957, 34.880...",Mississippi,01,Representative,Republican,"Nunnelee, Alan",https://www.congress.gov/member/alan-nunnelee/...,28,2801


In [261]:
data_df_2 = data_df.drop([26], axis = 0)

In [265]:
data_opd_final = data_df_2

In [42]:
# ProPublica data set
id = []
first_name = []
last_name = []
party = []
state = []
district_raw =[]
gender = []
date_of_birth = []
url = []
twitter_account = []

def proAPI():
	url = "https://api.propublica.org/congress/v1/115/house/members.json"
	r = requests.get(url,headers={"X-API-KEY":"Y4RTSJ8OEX4OAApedLDU4wwq2rLlSp36ln6eJXs1"})
	return r.content

def print_data(data):
	r = json.loads(data)
	members = r['results'][0]['members']
	for i in range(len(members)):
		id.append(members[i]['id'])
		first_name.append(members[i]['first_name'])
		last_name.append(members[i]['last_name'])
		party.append(members[i]['party'])
		state.append(members[i]['state'])
		district_raw.append(members[i]['district'])
		gender.append(members[i]['gender'])
		date_of_birth.append(members[i]['date_of_birth'])
		twitter_account.append(members[i]['twitter_account'])
		url.append(members[i]['url'])

if __name__ == '__main__':
	data = proAPI()
	print_data(data)


In [43]:
district_final = []
for val in district_raw:
    if len(val) == 1:
        district_final.append('0'+ val)
    else:
        district_final.append(val)


In [44]:
propublica_dict = {
    "first_name":first_name,
    "last_name":last_name, 
    "party":party, 
    "state_abbrev": state,
    "district": district_final,
    "gender":gender, 
    "date_of_birth":date_of_birth, 
    "twitter_account": twitter_account,
    "url":url
}

propublica_df = pd.DataFrame(propublica_dict)


In [25]:
propublica_df.loc[propublica_df['district'] == 'At-Large']

,first_name,last_name,party,state_abbrev,district,gender,date_of_birth,twitter_account,url
31,Lisa,Blunt Rochester,D,DE,At-Large,F,1962-02-10,RepLBR,https://bluntrochester.house.gov
33,Madeleine,Bordallo,D,GU,At-Large,F,1933-05-31,None,https://bordallo.house.gov
64,Liz,Cheney,R,WY,At-Large,F,1966-07-28,RepLizCheney,https://cheney.house.gov
89,Kevin,Cramer,R,ND,At-Large,M,1961-01-21,SenKevinCramer,https://www.cramer.senate.gov
150,Greg,Gianforte,R,MT,At-Large,M,1961-04-17,None,https://gianforte.house.gov
155,Jenniffer,González-Colón,R,PR,At-Large,F,1976-08-05,repjenniffer,https://gonzalez-colon.house.gov
300,Kristi,Noem,R,SD,At-Large,F,1971-11-30,RepKristiNoem,https://noem.house.gov
304,Eleanor,Norton,D,DC,At-Large,F,1937-06-13,EleanorNorton,https://norton.house.gov
324,Stacey,Plaskett,D,VI,At-Large,F,1966-05-13,staceyplaskett,https://plaskett.house.gov
334,Amata,Radewagen,R,AS,At-Large,F,1947-12-29,RepAmata,https://radewagen.house.gov


In [45]:
propublica_df['district'] = propublica_df['district'].replace({'At-Large': '00'})

In [47]:
propublica_merge_df = pd.merge(propublica_df, state_abbrev_df, on="state_abbrev")


In [49]:
propublica_merge_df['geoid'] = propublica_merge_df['state_code'] + propublica_merge_df['district']


In [205]:
#update_data = propublica_merge_df.set_index('twitter_account')

In [233]:
#update_data = update_data.reset_index(drop=False)

In [235]:
#update_data = update_data.drop([99, 416], axis = 0)

In [267]:
data_propublica_final = update_data

In [272]:
print(len(data_opd_final))
print(len(data_propublica_final))

435
435


In [278]:
raw_final_merge_df = data_propublica_final[['geoid', 'date_of_birth', 'first_name', 'last_name', 'gender', 'twitter_account']]

In [279]:
combined_data = pd.merge(data_opd_final, raw_final_merge_df, on = 'geoid')

In [283]:
print(len(combined_data))
combined_data.head(440)

435


,type,coordinates,state_label,district,rep_sen,party,name,url,state_code,geoid,date_of_birth,first_name,last_name,gender,twitter_account
0,Polygon,"[[[-88.202959, 35.008028], [-88.000032, 35.005...",Alabama,05,Representative,Republican,"Brooks, Mo",https://www.congress.gov/member/mo-brooks/B001274,01,0105,1954-04-29,Mo,Brooks,M,RepMoBrooks
1,Polygon,"[[[-112.147914, 33.304647], [-112.100437, 33.3...",Arizona,09,Representative,Democratic,"Sinema, Kyrsten",https://www.congress.gov/member/kyrsten-sinema...,04,0409,1976-07-12,Kyrsten,Sinema,F,SenatorSinema
2,MultiPolygon,"[[[[-122.520852, 37.594178], [-122.515326, 37....",California,14,Representative,Democratic,"Speier, Jackie",https://www.congress.gov/member/jackie-speier/...,06,0614,1950-05-14,Jackie,Speier,F,RepSpeier
3,Polygon,"[[[-123.056115, 39.048815], [-123.050849, 39.0...",California,05,Representative,Democratic,"Thompson, Mike",https://www.congress.gov/member/mike-thompson/...,06,0605,1951-01-24,Mike,Thompson,M,RepThompson
4,MultiPolygon,"[[[[-122.446316, 37.861046], [-122.441271, 37....",California,02,Representative,Democratic,"Huffman, Jared",https://www.congress.gov/member/jared-huffman/...,06,0602,1964-02-18,Jared,Huffman,M,RepHuffman
5,MultiPolygon,"[[[[-82.757334, 28.800728], [-82.751154, 28.80...",Florida,11,Representative,Republican,"Webster, Daniel",http://www.electwebster.com/,12,1211,1949-04-27,Daniel,Webster,M,RepWebster
6,Polygon,"[[[-83.226536, 32.584201], [-83.208715, 32.592...",Georgia,12,Representative,Republican,"Allen, Rick W.",https://www.congress.gov/member/rick-allen/A00...,13,1312,1951-11-07,Rick,Allen,M,reprickallen
7,Polygon,"[[[-87.920624, 41.889752], [-87.920566, 41.897...",Illinois,04,Representative,Democratic,"Gutierrez, Luis V.",https://www.congress.gov/member/luis-gutierrez...,17,1704,1953-12-10,Luis,Gutiérrez,M,RepGutierrez
8,Polygon,"[[[-85.911209, 41.261956], [-85.882318, 41.262...",Indiana,03,Representative,Republican,"Stutzman, Marlin A.",https://www.congress.gov/member/marlin-stutzma...,18,1803,1979-07-16,Jim,Banks,M,RepJimBanks
9,Polygon,"[[[-87.5314347879, 40.1483298719], [-87.531133...",Indiana,04,Representative,Republican,"Rokita, Todd",https://www.congress.gov/member/todd-rokita/R0...,18,1804,1970-02-09,Todd,Rokita,M,ToddRokita


In [284]:
def df_to_geojson(df, properties):
    
    ## df : the dataframe to convert to geojson
    ## properties : a list of columns in the dataframe to turn into geojson feature properties
   
    # create a new dictionary to contain properly formatted geojson data
    geojson = {'type':'FeatureCollection', 'features':[]}
 
    for x, row in df.iterrows():

        feature = {'type':'Feature',
                   'geometry':{'type': [],
                               'coordinates':[]},
                   'properties': {}}

        # fill in the type, coordinates
        feature['geometry']['type'] = row.type
        feature['geometry']['coordinates'] = row.coordinates

        # convert the array to a pandas.series
        geo_props = pd.Series(row)

        # for each column, get the value and add it as a new feature property
        # prop determines the list from the properties
        for prop in properties:

            geo_props[prop] = geo_props[prop]

            # now create a json format, here we have to make the dictionary properties
            feature['properties'][prop] = geo_props[prop]

        # add this feature to the list of features inside the dictionary
        geojson['features'].append(feature)
        
    return geojson


In [285]:
production_columns = ['state_label', 'district', 'rep_sen', 'party', 'name', 'first_name', 'last_name', 'gender', 'date_of_birth', 'url', 'twitter_account']
geojson_dict = df_to_geojson(combined_data, properties = production_columns)
geojson_export = json.dumps(geojson_dict, indent = 1)


In [286]:
# save the geojson result to a file
output_filename = 'Resources/production_export.geojson'
with open(output_filename, 'w') as production_file:
    json.dump(geojson_export, production_file, separators=(',', ': '))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))


435 geotagged features saved to file
